[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/planetlabs/notebooks/blob/master/jupyter-notebooks/analytics/quickstart/01_checking_available_feeds_and_subscriptions.ipynb)

###  Planet Analytics API Tutorial Part 1

# Summarizing Feeds and Subscriptions

Planet Analytics leverages computer vision to transform our imagery into Analytic Feeds that detect and classify objects, identify geographic features, and understand change over time across the globe. 

Users have subscriptions to feeds in a specific Area of Interest (AOI) and Time Interval of Interest (TOI). For example, a subscription could be Road Detection over 12 months in San Francisco, California. Users can get a list of their subscriptions from the Analytics API.

This notebook demonstrates how to describe available [Analytics Feeds](https://developers.planet.com/docs/analytics/#analytic-feeds) and [Subscriptions](https://developers.planet.com/docs/analytics/#subscriptions) with the Planet Analytics API.

## Setup
To use this notebook, you need to have the following:
- A Planet account with access to the [Analytics API](https://developers.planet.com/docs/analytics/)
- A Planet [API Key](https://developers.planet.com/quickstart/apis/)

#### Import Packages

In [1]:
import os
import requests
from pprint import pprint
import pandas as pd
from ipyleaflet import Map, GeoJSON

#### Set API Key

In [2]:
# If your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.environ.get('PL_API_KEY', 'PASTE_YOUR_KEY_HERE')

# Alternatively, you can just set your API key directly as a string variable:
# API_KEY = "YOUR_PLANET_API_KEY_HERE"
# Construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

#### Set the base url for the Planet Analytic Feeds product
See the [Analytics API Docs](https://developers.planet.com/docs/analytics/) for more details.

In [3]:
BASE_URL = "https://api.planet.com/analytics/"

#### Test API Connection

In [4]:
feed_list_url = f'{BASE_URL}feeds?limit=1'
resp = requests.get(feed_list_url, auth=BASIC_AUTH)
if resp.status_code == 200:
    print('Yay, you can access the Analytics API')
else:
    print('Something is wrong:', resp.content)

Yay, you can access the Analytics API


## Summarizing Feeds
In this section, we will see describe the available feeds.

#### How many feeds are there?

In [5]:
limit = 1000
feed_list_url = f'{BASE_URL}feeds?limit={limit}'
pprint(f'Feeds endpoint: {feed_list_url}')

# Send a GET request to the feeds endpoint, using basic authentication
resp = requests.get(feed_list_url, auth=BASIC_AUTH)
feeds = resp.json()['data']
feed_count = len(feeds)
pprint(f'Available feeds: {feed_count}')

# Check if the available feeds count is equal to the specified limit
if feed_count >= limit:
    print('More feeds are probably available through pagination links')
    print(resp.json()['links'])

'Feeds endpoint: https://api.planet.com/analytics/feeds?limit=1000'
'Available feeds: 158'


#### Inspecting feed metadata

In [6]:
pprint(feeds[0])

{'created': '2023-06-13T16:01:15.186Z',
 'customer_contract': False,
 'description': 'Version semantics: v{source mosaic}.{G4 process}',
 'events': {},
 'id': '5b73a193-484f-4aca-961b-2b1b21032a12',
 'interval': 120,
 'links': [{'href': 'https://api.planet.com/analytics/feeds/5b73a193-484f-4aca-961b-2b1b21032a12',
            'rel': 'self',
            'type': 'application/json'},
           {'href': 'https://api.planet.com/analytics/feeds',
            'rel': 'feeds',
            'type': 'application/json'}],
 'outputClasses': [],
 'process': {'actions': [{'Fn::AddTask': ['IA_land_cover_change_v1_9_3',
                                          {'annotations': {'results': 'object_detections_metadata:files/object_detections.geojson'},
                                           'args': {'alpha_channel_band': 3,
                                                    'cell_size': 8,
                                                    'input_segmentations_from_g4_ops': False,
                 

**Some of the fields include:**
- id: this is a unique identifier for a feed
- title: a human friendly name for the feed
- description: more detail text about the feed
- created: timestamp for when the feed was originally created
- updated: timestamp for when the feed was last modified
- source: a blob describing the imagery on which the feed is based
- target: a blob describing the feed's output format
- links: a list of blobs containing urls to related resources

#### Showing all available feeds in a table
We can use a pandas DataFrame to summarize the available feeds.

In [7]:
# Bump this up in case there are many available feeds to display
pd.options.display.max_rows = 1000
# Make a dataframe from the feeds json data
df = pd.DataFrame(feeds)

# Instead of including the entire source and target dicts, make columns for the types
df['targetType'] = ""
df['sourceType'] = ""

for row in range(0, len(df)):
    df.loc[:, ('targetType', row)] = df['target'][row]['type']
    df.loc[:, ('sourceType', row)] = df['source'][row][0]['type']

df[['id', 'title', 'description', 'sourceType', 'targetType', 'created', 'updated']]


,id,title,description,sourceType,targetType,created,updated
0,5b73a193-484f-4aca-961b-2b1b21032a12,Road change - weekly cadence - lookback 2023-0...,Version semantics: v{source mosaic}.{G4 process},mosaic,collection,2023-06-13T16:01:15.186Z,2023-06-13T16:01:15.186Z
1,6ca29572-1f8f-4d30-8935-1ed21e088c96,Building change - weekly cadence - lookback 20...,Version semantics: v{source mosaic}.{G4 process},mosaic,collection,2023-06-13T16:00:26.951Z,2023-06-13T16:00:26.951Z
2,c97c8c75-27c9-401d-a8b5-2bbc52612f42,Building change - weekly cadence - lookback 20...,Version semantics: v{source mosaic}.{G4 process},mosaic,collection,2023-06-07T19:02:42.012Z,2023-06-07T19:02:42.012Z
3,4088eddd-7aa6-4766-8be1-70d1a579de20,Building change - weekly cadence - lookback 20...,Version semantics: v{source mosaic}.{G4 process},mosaic,collection,2023-06-07T18:41:16.285Z,2023-06-07T18:41:16.285Z
4,6261fea4-dbd0-4399-b256-254c93eb2c95,Road change - weekly cadence - lookback 2020-0...,Version semantics: v{source mosaic}.{G4 process},mosaic,collection,2023-06-07T18:38:14.382Z,2023-06-07T18:38:14.382Z
5,842b0912-45ca-4ef0-b100-1ef1685a74ef,Ship Detection TOA - 50m+,Ship Detection TOA -50m+,mosaic,collection,2023-05-04T20:27:30.719Z,2023-05-04T20:27:30.719Z
6,8573bae0-e718-42c5-af82-31bd7c3fbbc6,Ship Detection TOA - 25m+,Ship Detection TOA - 25m+,mosaic,collection,2023-05-02T20:53:00.934Z,2023-05-02T20:53:00.934Z
7,b644c456-a67d-4ff0-9b34-b2c073193860,Road state - weekly cadence - lookback 2021-09...,Version semantics: v{source mosaic}.{G4 process},mosaic,collection,2023-03-14T22:16:41.793Z,2023-03-14T22:16:41.793Z
8,fa5d9831-aa30-4507-96da-25cd6aacdf5a,Copy of Ship Detections,Ship detections from rectified PlanetScope ima...,mosaic,collection,2023-02-23T19:05:31.179Z,2023-02-23T19:05:31.179Z
9,a069620d-d076-4eb3-8326-0a155b510697,Building change - monthly cadence - lookback 2...,Version semantics: v{source mosaic}.{G4 process},mosaic,collection,2023-01-20T17:57:45.802Z,2023-01-20T17:57:45.802Z


## Summarizing Subscriptions
Now that we know about available feeds, let's check out available subscriptions.

In [8]:
limit = 1000
subscriptions_url = f'{BASE_URL}subscriptions?limit={limit}'
print(f'Subscriptions endpoint: {subscriptions_url}')
resp = requests.get(subscriptions_url, auth=BASIC_AUTH)
subs = resp.json()['data']
sub_count = len(subs)
print(f'Available subscriptions: {sub_count}')
if sub_count >= limit:
    print('More subscriptions are probably available through pagination links')
    print(resp.json()['links'])

Subscriptions endpoint: https://api.planet.com/analytics/subscriptions?limit=1000
Available subscriptions: 562


#### What's in a subscription?

In [9]:
pprint(subs[0])

{'created': '2023-07-05T20:42:11.683Z',
 'description': 'Ship Detection 50m+',
 'endTime': '2023-06-30T00:00:00.000Z',
 'feedID': '5b314bf0-ae01-4c98-971f-7c806f8637f0',
 'geometry': {'coordinates': [[[-69.010057, 63.781113],
                               [-68.127523, 63.186934],
                               [-67.538771, 63.010675],
                               [-66.908122, 62.647522],
                               [-66.081585, 62.212382],
                               [-65.813363, 62.0817],
                               [-65.964955, 61.962506],
                               [-66.28557, 61.827582],
                               [-67.000334, 61.430995],
                               [-66.307191, 60.70831],
                               [-63.108471, 60.88852],
                               [-61.045564, 63.079567],
                               [-64.345754, 62.47372],
                               [-65.252948, 62.643302],
                               [-65.74745, 63.118],


Subscriptions also have ID, title, description, created, and updated fields.
Additionally, there are fields for:
- feedID: which feed this subscription is for
- startTime: timestamp for the subscription's beginning
- endTime: timestamp for the subscription's ending
- geometry: spatial area of interest to which the subscription has access

**Important:** 
Subscriptions will only get results for source imagery observed between the `startTime` and `endTime` within the specified `geometry`.

`created` and `updated` refer to when the subscription itself was set up or modified and **do not** impact results that show up for the subscription. 

`startTime` and `endTime` **do** limit feed results for the subscription.

#### Showing all available subscriptions

In [10]:
df = pd.DataFrame(subs)
df[['id', 'title', 'description', 'feedID', 'startTime', 'endTime', 'created', 'updated']]

,id,title,description,feedID,startTime,endTime,created,updated
0,74e7c01e-f25c-4712-9c72-f5b58afb6f53,Eastern Canada,Ship Detection 50m+,5b314bf0-ae01-4c98-971f-7c806f8637f0,2023-06-01T00:00:00.000Z,2023-06-30T00:00:00.000Z,2023-07-05T20:42:11.683Z,2023-07-05T20:42:11.683Z
1,d113c5ac-6d64-4f7b-b501-16fa45a49293,Weekly road detections over Nairobi,https://hello.planet.com/jira/browse/AN-6006,04fa55c5-ff2c-470e-a6fc-2665c53dd294,2022-01-01T00:00:00.000Z,2023-06-15T00:00:00.000Z,2023-06-15T20:00:54.197Z,2023-06-15T20:00:54.197Z
2,ea0b8c2b-7504-4018-bed3-2ce089dd77c5,Weekly building detections over Nairobi,https://hello.planet.com/jira/browse/AN-6006,f1801c8d-6d18-47eb-8d36-d02355525483,2022-01-01T00:00:00.000Z,2023-06-15T00:00:00.000Z,2023-06-15T19:59:42.938Z,2023-06-15T19:59:42.938Z
3,a7011f4c-20ec-4344-95fa-24708bee57e1,Weekly road change detection over Nairobi,https://hello.planet.com/jira/browse/AN-6006,f2ad11d2-ce10-497a-b958-9aeea828f3d2,2022-01-01T00:00:00.000Z,2023-06-15T00:00:00.000Z,2023-06-15T19:55:57.795Z,2023-06-15T19:55:57.795Z
4,7dd2c0af-d88e-471d-bc76-9ffdb931ed6a,Weekly building change detection over Nairobi,https://hello.planet.com/jira/browse/AN-6006,5be42113-29ea-4be0-a029-2b5161a8662f,2022-01-01T00:00:00.000Z,2023-06-15T00:00:00.000Z,2023-06-15T19:54:56.334Z,2023-06-15T19:56:14.977Z
5,520ca04e-cef3-4da7-8187-165059a57ff1,San Diego Area,Ship Detection (50m+),5b314bf0-ae01-4c98-971f-7c806f8637f0,2023-01-01T00:00:00.000Z,2023-04-01T00:00:00.000Z,2023-06-15T12:30:09.880Z,2023-06-15T12:30:09.880Z
6,9ad3f8ac-d8f5-47db-8295-096097d2f6a7,"Spokane, WA",Weekly Road Change Detection,5b73a193-484f-4aca-961b-2b1b21032a12,2023-06-01T00:00:00.000Z,2024-07-31T00:00:00.000Z,2023-06-13T16:03:48.843Z,2023-06-13T19:41:35.321Z
7,9a53b5f4-f89e-4de4-9c7a-1e01fa3dc462,"Spokane, WA",Weekly Building Change Detection,6ca29572-1f8f-4d30-8935-1ed21e088c96,2023-06-01T00:00:00.000Z,2024-07-31T00:00:00.000Z,2023-06-13T16:02:39.811Z,2023-06-13T19:41:23.121Z
8,508b6844-5933-4810-a581-0cb18b37ce8b,"Spokane, WA",Weekly Road Detection,04fa55c5-ff2c-470e-a6fc-2665c53dd294,2023-06-01T00:00:00.000Z,2024-07-31T00:00:00.000Z,2023-06-13T15:09:58.454Z,2023-06-13T19:41:10.779Z
9,d0dda157-6d3b-4120-99a7-a9dd9fce0d1a,"Spokane, WA",Weekly Building Detection,f1801c8d-6d18-47eb-8d36-d02355525483,2023-06-01T00:00:00.000Z,2024-07-31T00:00:00.000Z,2023-06-13T15:08:58.379Z,2023-06-13T19:40:57.940Z


#### Filtering subscriptions by feed

In [11]:
feed_id = feeds[0]['id']
feed_title = feeds[0]['title']
print(feed_title)
print('id:', feed_id)

Road change - weekly cadence - lookback 2023-03-20 - v0.25
id: 5b73a193-484f-4aca-961b-2b1b21032a12


In [12]:
filtered_subscriptions_url = f'{BASE_URL}subscriptions?feedID={feed_id}'
print('url:', filtered_subscriptions_url)
resp = requests.get(filtered_subscriptions_url, auth=BASIC_AUTH)
filtered_subs = resp.json()['data']
filtered_sub_count = len(filtered_subs)
print(f'You have access to {filtered_sub_count} subscriptions for feed {feed_id} ({feed_title})')

url: https://api.planet.com/analytics/subscriptions?feedID=5b73a193-484f-4aca-961b-2b1b21032a12
You have access to 1 subscriptions for feed 5b73a193-484f-4aca-961b-2b1b21032a12 (Road change - weekly cadence - lookback 2023-03-20 - v0.25)


#### Inspecting a subscription's geometry
Subscriptions have a spatial area of interest described by a geojson geometry. We can visualize the area of interest for a subscription on a map.

In [13]:
# Get the latest subscription's geometry
subscription = subs[0]
geom = subscription['geometry']
pprint(geom)

{'coordinates': [[[-69.010057, 63.781113],
                  [-68.127523, 63.186934],
                  [-67.538771, 63.010675],
                  [-66.908122, 62.647522],
                  [-66.081585, 62.212382],
                  [-65.813363, 62.0817],
                  [-65.964955, 61.962506],
                  [-66.28557, 61.827582],
                  [-67.000334, 61.430995],
                  [-66.307191, 60.70831],
                  [-63.108471, 60.88852],
                  [-61.045564, 63.079567],
                  [-64.345754, 62.47372],
                  [-65.252948, 62.643302],
                  [-65.74745, 63.118],
                  [-67.94105, 63.784663],
                  [-69.010057, 63.781113]]],
 'type': 'Polygon'}


In [14]:
# Make a map, and draw the subscription geometry
lon = geom['coordinates'][0][0][0]
lat = geom['coordinates'][0][0][0]

m = Map(center=(lat, lon), zoom=8)
geo_json = GeoJSON(data=subscription['geometry'], style = {'color': 'blue', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
m.add_layer(geo_json)
m

Map(center=[-69.010057, -69.010057], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…